In [24]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cmake
import face_recognition as fr
import cv2 
import csv
from datetime import datetime

In [25]:
path='photo_attd'

In [26]:
os.listdir(path)

['ashlin.jpg']

In [27]:
mylist=os.listdir(path)
mylist

['ashlin.jpg']

In [28]:
imgs=[]
classnames=[]
for i in mylist:
    imgpath=os.path.join(path,i)
    current_img=cv2.imread(imgpath)
    print(current_img)
    imgs.append(current_img)
    classnames.append(i.split(".")[0])

[[[ 80 122 127]
  [ 79 121 126]
  [ 78 120 125]
  ...
  [149 167 220]
  [151 167 220]
  [151 167 220]]

 [[ 82 124 129]
  [ 82 124 129]
  [ 81 123 128]
  ...
  [149 167 220]
  [151 167 220]
  [151 167 220]]

 [[ 84 126 131]
  [ 84 126 131]
  [ 84 126 131]
  ...
  [149 167 220]
  [151 167 220]
  [151 167 220]]

 ...

 [[177 202 222]
  [177 202 222]
  [177 202 222]
  ...
  [253 233 228]
  [253 233 228]
  [253 233 228]]

 [[177 202 222]
  [177 202 222]
  [177 202 222]
  ...
  [253 233 228]
  [253 233 228]
  [253 233 228]]

 [[177 202 222]
  [177 202 222]
  [177 202 222]
  ...
  [253 233 228]
  [253 233 228]
  [254 234 229]]]


In [29]:
#facelocation and encoding
def faceencodings(images):
    encodelist=[]
    for img in images:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        face_in_frame=fr.face_locations(img)
        print(face_in_frame)
        encode=fr.face_encodings(img,face_in_frame)[0]
        encodelist.append(encode)
    return encodelist

In [30]:
encode_unkown_face=faceencodings(imgs)
encode_unkown_face

[(333, 632, 718, 247)]


[array([-0.10591254,  0.16010793, -0.01697507, -0.07006706, -0.01190534,
        -0.03225174,  0.00695118, -0.09759618,  0.24466833, -0.08411171,
         0.19911148,  0.04648799, -0.24028169, -0.05231431, -0.05049918,
         0.10878997, -0.23776734, -0.11199388, -0.1058192 , -0.06632022,
         0.12722623,  0.07323461,  0.10960414,  0.03950097, -0.15481621,
        -0.33704695, -0.12384219, -0.0860009 ,  0.05592302, -0.16207497,
         0.03914886, -0.03776372, -0.20363021, -0.04547618, -0.03445619,
         0.01756778, -0.03579126, -0.07489832,  0.16911356,  0.04463591,
        -0.20938928, -0.02609676,  0.001589  ,  0.25987077,  0.19419803,
         0.06079314, -0.01614029, -0.11499977,  0.03102693, -0.22097246,
         0.05532886,  0.108616  ,  0.07551902,  0.03078453,  0.03655342,
        -0.1099256 ,  0.00564568,  0.08193651, -0.20897894,  0.00510226,
         0.04770337, -0.08337816, -0.00364109,  0.0348384 ,  0.1752127 ,
         0.08488197, -0.13761638, -0.08819197,  0.1

In [31]:
video=cv2.VideoCapture(0)
while True:
    suc,img=video.read()
    img1=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    face_in_frame=fr.face_locations(img1)
    encode_frame=fr.face_encodings(img1,face_in_frame)

    for face_loc,enc in zip(face_in_frame,encode_frame):
        matches=fr.compare_faces(encode_unkown_face,enc)
        face_distance=fr.face_distance(encode_unkown_face,enc)
    
        matchindex=np.argmin(face_distance)

        if matches[matchindex]==True:
            name=classnames[matchindex]
            print(name)
            y1,x2,y2,x1=face_loc
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)

    cv2.imshow("Webcam",img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

video.release()
cv2.destroyAllWindows()

ashlin
ashlin
ashlin
ashlin


In [32]:
def markAttendance(name):
    filename = 'attendance.csv'

    # Create file if not exists
    if not os.path.exists(filename):
        with open(filename, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Name', 'Date', 'Time'])

    with open(filename, 'r+', newline='') as f:
        reader = csv.reader(f)
        name_list = [row[0] for row in reader]

        if name not in name_list:
            now = datetime.now()
            date = now.strftime('%Y-%m-%d')
            time = now.strftime('%H:%M:%S')
            writer = csv.writer(f)
            writer.writerow([name, date, time])


In [33]:
video = cv2.VideoCapture(0)

while True:
    suc, img = video.read()
    img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    face_in_frame = fr.face_locations(img1)
    encode_frame = fr.face_encodings(img1, face_in_frame)

    for face_loc, enc in zip(face_in_frame, encode_frame):
        matches = fr.compare_faces(encode_unkown_face, enc)
        face_distance = fr.face_distance(encode_unkown_face, enc)

        matchindex = np.argmin(face_distance)

        if matches[matchindex]:
            name = classnames[matchindex].upper()
            markAttendance(name)   # ✅ Mark attendance

            y1, x2, y2, x1 = face_loc
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, name, (x1, y2 + 25),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Webcam", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()
